In [1]:
import pandas as pd 
import numpy as np 

In [2]:
# load data

# updated data with 25% identity and 2.0 Angstrom cutoffs 
ss_2022_25_20 = pd.read_csv('2022-08-06-pdb-intersect-pisces_pc25_r2.0.csv')

# updated data with 25% identity and 2.5 Angstrom cutoffs 
ss_2022_25_25 = pd.read_csv('2022-08-06-pdb-intersect-pisces_pc25_r2.5.csv')

# updated data with 30% identity and 2.5 Angstrom cutoffs 
ss_2022_30_25 = pd.read_csv('2022-08-06-pdb-intersect-pisces_pc30_r2.5.csv')

In [3]:
# split the data
from sklearn.model_selection import train_test_split

# start by making an 80/20 split to obtain the training data
X_train, X_test, y_train, y_test = train_test_split(ss_2022_30_25['seq'], ss_2022_30_25[['sst3','sst8']], test_size = 0.2, random_state = 42)
# then do another 50/50 split on the "test" data obtained in the 
# previous line to get the 10% test sample and the 10% validation
# sample
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=383)

print('There are %s records in X_train' %len(X_train))
print('There are %s records in X_valid' %len(X_valid))
print('There are %s records in X_test' %len(X_test))


There are 10724 records in X_train
There are 1341 records in X_valid
There are 1341 records in X_test


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# convert each sequence to an array of k-mers (i.e. n-grams)
# using k-mers allows us to include more information about the context 
# in which each amino acid appears within the primary protein structure
def seq2kmers(seqs, n = 3):
    return np.array([[seq[i:i+n] for i in range(len(seq))] for seq in seqs], dtype = object)

train_kmers = seq2kmers(X_train, n = 3)
valid_kmers = seq2kmers(X_valid, n = 3)
test_kmers  = seq2kmers(X_test, n=3)

# find the maximum sequence length across all three folds
# of the dataset (we need to use this to pad the sequences
# later on)
maxlen = 0
for kmers in [train_kmers, valid_kmers, test_kmers]:
    for seq in train_kmers:
        if len(seq) > maxlen:
            maxlen = len(seq)

# convert the k-mers to encoded sequences
encoder = Tokenizer()
encoder.fit_on_texts(train_kmers)
train_sequences = encoder.texts_to_sequences(train_kmers)
train_sequences = sequence.pad_sequences(train_sequences, maxlen = maxlen, padding = 'post')

test_sequences = encoder.texts_to_sequences(test_kmers)
test_sequences = sequence.pad_sequences(test_sequences, maxlen=maxlen, padding='post')

valid_sequences = encoder.texts_to_sequences(valid_kmers)
valid_sequences = sequence.pad_sequences(valid_sequences, maxlen=maxlen, padding='post')

# encode the target sequences 
y = {}

# convert training, validation, and test SSTs for both SST3 and SST8 into
# numeric sequences that can be used by the model.
for target in ['sst3', 'sst8']:
    y[target] = {}
    decoder = Tokenizer(char_level = True)
    decoder.fit_on_texts(y_train[target])

    # convert texts to sequences 
    y_train_sequences = decoder.texts_to_sequences(y_train[target])
    # pad the sequences so that they are all the same length (we
    # use the length of the longest sequence to avoid truncation)
    y_train_sequences = sequence.pad_sequences(y_train_sequences, maxlen = maxlen, padding = 'post')
    # convert the sequences to categorical tensors
    y_train_sequences = to_categorical(y_train_sequences)

    # repeat this process on the test labels 
    y_test_sequences = decoder.texts_to_sequences(y_test[target])
    y_test_sequences = sequence.pad_sequences(y_test_sequences, maxlen = maxlen, padding = 'post')
    y_test_sequences = to_categorical(y_test_sequences)

    # repeat the process on the validation labels
    y_valid_sequences = decoder.texts_to_sequences(y_valid[target])
    y_valid_sequences = sequence.pad_sequences(y_valid_sequences, maxlen = maxlen, padding = 'post')
    y_valid_sequences = to_categorical(y_valid_sequences)

    # store the sequences and model metadata in a dict
    y[target]['train_sequences'] = y_train_sequences
    y[target]['valid_sequences'] = y_valid_sequences 
    y[target]['test_sequences'] = y_test_sequences 
    y[target]['n_words'] = len(encoder.word_index) + 1 
    y[target]['n_ssts'] = len(decoder.word_index) + 1


In [5]:
# Build the model for predicting the SST3 sequence

from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional


model = Sequential([
    # the embedding layer converts the input sequences into higher-dimensional embeddings
    # that will be fed to the LSTM layer 
    Embedding(input_dim = y['sst3']['n_words'], output_dim = 256, input_length = maxlen),
    # the LSTM layer processes the sequences, retaining relevant information about the
    # context of each input that it processes from both earlier and later in the sequence 
    Bidirectional(LSTM(units = 64, return_sequences = True, recurrent_dropout = 0.1)),
    # this layer converts the output from the LSTM layer into the output sequence
    TimeDistributed(Dense(y['sst3']['n_ssts'], activation = 'softmax'))])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2128, 256)         2176512   
                                                                 
 bidirectional (Bidirectiona  (None, 2128, 128)        164352    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 2128, 4)          516       
 ibuted)                                                         
                                                                 
Total params: 2,341,380
Trainable params: 2,341,380
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Train the model
from sklearn.model_selection import train_test_split
from keras import backend as K

# Q3 Accuracy Implementation from https://www.kaggle.com/code/helmehelmuto/secondary-structure-prediction-with-keras/notebook
# "SS prediction is usually evaluated by Q3 or Q8 accuracy, which measures the percent of residues for which 3-state or 8-state 
# secondary structure is correctly predicted"  (doi: 10.1038/srep18962)
def q3_acc(y_true, y_pred):
    y = tf.argmax(y_true, axis=-1)
    y_ = tf.argmax(y_pred, axis=-1)
    mask = tf.greater(y, 0)
    return K.cast(K.equal(tf.boolean_mask(y, mask), tf.boolean_mask(y_, mask)), K.floatx())

model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy", metrics = ["accuracy", q3_acc])
model.fit(train_sequences, 
          y['sst3']['train_sequences'], 
          batch_size = 128, 
          epochs = 10, 
          validation_data = (valid_sequences, 
                             y['sst3']['valid_sequences']), 
          verbose = 1)




Epoch 1/10


2023-01-31 20:08:44.183129: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 2/84 [..............................] - ETA: 7:04 - loss: 1.2134 - accuracy: 0.4696 - q3_acc: 0.2160

KeyboardInterrupt: 